### Advanced Numpy

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

#### Block of memory

In [ ]:
>>> x = np.array([1, 2, 3], dtype=np.int32)
>>> x.data
<... at ...>


In [ ]:
>>> bytes(x.data)
'\x01\x00\x00\x00\

In [ ]:
>>> x.__array_interface__['data'][0]

In [ ]:
>>> x.__array_interface__

In [ ]:
>>> x = np.array([1, 2, 3, 4])
>>> y = x[:-1]
>>> x[0] = 9
>>> y
array([

In [ ]:
>>> y = np.frombuffer(x, dtype=np.int8)
>>> y.data
<... at ...>


In [ ]:
>>> y.base is x
True


In [ ]:
>>> y.flags
C_CONTIGUOUS : True
F_CONTIGUOUS : True
OWNDATA : False
WRITEABLE : False
ALIGNED : True
WRITEBACKIFCOPY : False
UPDATEIFCOPY : False

#### Data types

In [ ]:
>>> np.dtype(int).type
<type 'numpy.int64'>


In [ ]:
>>> np.dtype(int).itemsize
8


In [ ]:
>>> np.dtype(int).byteorder

In [ ]:
>>> wav_header_dtype = np.dtype([
... ("chunk_id", (bytes, 4)), # flexible-sized scalar type, item size 4
... ("chunk_size", "<u4"), # little-endian unsigned 32-bit integer
... ("format", "S4"), # 4-byte string
... ("fmt_id", "S4"),
... ("fmt_size", "<u4"),
... ("audio_fmt", "<u2"), #
... ("num_channels", "<u2"), # .. more of the same ...
... ("sample_rate", "<u4"), #
... ("byte_rate", "<u4"),
... ("block_align", "<u2"),
... ("bits_per_sample", "<u2"),
... ("data_id", ("S1", (2, 2))), # sub-array, just for fun!
... ("data_size", "u4"),
... #
... # the sound data itself cannot be represented here:
... #

In [ ]:
wav_header_dtype = np.dtype(dict(
... names=['format', 'sample_rate', 'data_id'],
... offsets=[offset_1, offset_2, offset_3], # counted from start of structure in bytes
... formats=list of dtypes for each of the fields,
... ))

In [ ]:
>>> f = open('data/test.wav', 'r')
>>> wav_header = np.fromfile(f, dtype=wav_header_dtype, count=1)
>>> f.close()
>>> print(wav_header)


In [ ]:
[ ('RIFF', 17402L, 'WAVE', 'fmt ', 16L, 1, 1, 16000L, 32000L, 2, 16, [['d', 'a'], ['t', 'a']],␣
˓→17366L)]
>>> wav_header['sample_rate']

In [ ]:
>>> wav_header['data_id']
array([[['d', 'a'],
['t', 'a']]],

In [ ]:
>>> wav_header.shape

In [ ]:
(1,)
>>> wav_header['data_id'].shape

In [ ]:
>>> x = np.array([1, 2, 3, 4], dtype=np.float)
>>> x
array([1., 2., 3., 4.])


In [ ]:
>>> y = x.astype(np.int8)
>>> y
array([1, 2, 3, 4], dtype=int8)


In [ ]:
>>> y + 1
array([2, 3, 4, 5], dtype=int8)


In [ ]:
>>> y + 256
array([257, 258, 259, 260], dtype=int16)


In [ ]:
>>> y + 256.0
array([257., 258., 259., 260.])
>>> y + np.array([256], dtype=np.int32)
array([257, 258, 259, 260], dtype=int32)

In [ ]:
>>> y + 256.0
array([257., 258., 259., 260.])


In [ ]:
>>> y + np.array([256], dtype=np.int32)
array([257, 258, 259, 260], dtype=int32)

In [ ]:
>>> y[:] = y + 1.5
>>> y
array([2, 3, 4, 5], dtype=int8)

In [ ]:
>>> x = np.array([1, 2, 3, 4], dtype=np.uint8)
>>> x.dtype = "<i2"
>>> x
array([ 513, 1027], dtype=int16)


In [ ]:
>>> 0x0201, 0x0403
(513, 1027)

In [ ]:
>>> y = x.view("<i4")
>>> y
array([67305985], dtype=int32)


In [ ]:
>>> 0x04030201
67305985

In [ ]:
>>> x[1] = 5
>>> y
array([

In [ ]:
>>> y.base is x

In [ ]:
>>> x = np.zeros((10, 10, 4), dtype=np.int8)
>>> x[:, :, 0] = 1
>>> x[:, :, 1] = 2
>>> x[:, :, 2] = 3
>>> x[:, :, 3] = 4

In [ ]:
>>> y = ...
>>> assert (y['r'] == 1).all()
>>> assert (y['g'] == 2).all()
>>> assert (y['b'] == 3).all()
>>> assert (y['a'] == 4).all()

In [ ]:
>>> y = x.view([('r', 'i1'),
... ('g', 'i1'),
... ('b', 'i1'),
... ('a', 'i1')]
... )[:, :, 0]

In [ ]:
>>> y = np.array([[1, 3], [2, 4]], dtype=np.uint8).transpose()
>>> x = y.copy()
>>> x
array([[1, 2],
[3, 4]], dtype=uint8)


In [ ]:
>>> y
array([[1, 2],
[3, 4]], dtype=uint8)


In [ ]:
>>> x.view(np.int16)
array([[ 513],
[1027]], dtype=int16)


In [ ]:
>>> 0x0201, 0x0403
(513, 1027)


In [ ]:
>>> y.view(np.int16)
array([[ 769, 1026]], dtype=int16)

In [ ]:
>>> 0x0301, 0x0402
(769, 1026)

#### Indexing scheme: strides

In [ ]:
>>> x = np.array([[1, 2, 3],
... [4, 5, 6],
... [7, 8, 9]], dtype=np.int8)


In [ ]:
>>> x.tobytes('A')
b'\x01\x02\x03\x04\x05\x06\x07\x08\t'

In [ ]:
>>> x.strides
(3, 1)

In [ ]:

>>> byte_offset = 3*1 + 1*2 # to find x[1, 2]
>>> x.flat[byte_offset]

In [ ]:
6
>>> x[1, 2]

In [ ]:
>>> x = np.array([[1, 2, 3],
... [4, 5, 6]], dtype=np.int16, order='C')


In [ ]:
>>> x.strides
(6, 2)


In [ ]:
>>> x.tobytes('A')
b'\x01\x00\x02\x00\

In [ ]:
>>> y = np.array(x, order='F')
>>> y.strides

In [ ]:
>>> y.tobytes('A')

In [ ]:
>>> y = np.array([[1, 3], [2, 4]], dtype=np.uint8).transpose()
>>> x = y.copy()

In [ ]:
>>> x.strides
(2, 1)


In [ ]:
>>> y.strides
(1, 2)

In [ ]:
>>> x.tobytes('A')
b'\x01\x02\x03\x04'


In [ ]:
>>> y.tobytes('A')
b'\x01\x03\x02\x04'

In [ ]:
>>> x = np.array([1, 2, 3, 4, 5, 6], dtype=np.int32)
>>> y = x[::-1]
>>> y
array([6, 5, 4, 3, 2, 1], dtype=int32)


In [ ]:
>>> y.strides
(-4,)


In [ ]:
>>> y = x[2:]
>>> y.__array_interface__['data'][0] - x.__array_interface__['data'][0]
8


In [ ]:
>>> x = np.zeros((10, 10, 10), dtype=np.float)
>>> x.strides
(800, 80, 8)

In [ ]:
>>> x[::2,::3,::4].strides

In [ ]:
>>> x = np.zeros((10, 10, 10), dtype=np.float)
>>> x.strides
(800, 80, 8)


In [ ]:
>>> x.T.strides
(8, 80, 800)
But: not all reshaping

In [ ]:
>>> a = np.arange(6, dtype=np.int8).reshape(3, 2)
>>> b = a.T
>>> b.strides
(1, 2)
So far, so

In [ ]:
>>> bytes(a.data)
b'\x00\x01\x02\x03\x04\x05'


In [ ]:
>>> b
array([[0, 2, 4],
[1, 3, 5]], dtype=int8)


In [ ]:
>>> c = b.reshape(3*2)
>>> c
array([0, 2, 4, 1, 3, 5], dtype=int8)

In [ ]:
>>> from numpy.lib.stride_tricks import as_strided
>>> help(as_strided)
as_strided(x, shape=None,

In [ ]:
>>> x = np.array([1, 2, 3, 4], dtype=np.int16)
>>> as_strided(x, strides=(2*2, ), shape=(2, ))
array([1, 3], dtype=int16)
>>> x[::2]
array([1, 3], dtype=int16)

In [ ]:
# Exercise

array([1, 2, 3, 4], dtype=np.int8)
-> array([[1, 2, 3, 4],
[1, 2, 3, 4],
[1, 2, 3, 4]], dtype=np.int8)

In [ ]:
>>> x = np.array([1, 2, 3, 4], dtype=np.int8)
>>> y = as_strided(x, strides=(0, 1), shape=(3, 4))
>>> y
array([[1, 2, 3, 4],
[1, 2, 3, 4],
[1, 2, 3, 4]], dtype=int8)
>>> y.base.base is x
True

In [ ]:
# Broadcasting
>>> x = np.array([1, 2, 3, 4], dtype=np.int16)
>>> x2 = as_strided(x, strides=(0, 1*2), shape=(3, 4))
>>> x2
array([[1, 2, 3, 4],
[1, 2, 3, 4],
[1, 2, 3, 4]], dtype=int16)

In [ ]:
>>> y = np.array([5, 6, 7], dtype=np.int16)
>>> y2 = as_strided(y, strides=(1*2, 0), shape=(3, 4))
>>> y2
array([[5, 5, 5, 5],
[6, 6, 6, 6],
[7, 7, 7, 7]], dtype=int16)

In [ ]:
>>> x2 * y2
array([[ 5, 10, 15, 20],
[ 6, 12, 18, 24],
[ 7, 14, 21, 28]], dtype=int16)

In [ ]:
>>> x = np.array([1, 2, 3, 4], dtype=np.int16)
>>> y = np.array([5, 6, 7], dtype=np.int16)
>>> x[np.newaxis,:] * y[:,np.newaxis]
array([[ 5, 10, 15, 20],
[ 6, 12, 18, 24],
[ 7, 14, 21, 28]], dtype=int16)

In [ ]:
>>> x = np.array([[1, 2, 3],
... [4, 5, 6],
... [7, 8, 9]], dtype=np.int32)
>>> x_diag = as_strided(x, shape=(3,), strides=(???,))

In [ ]:
# Solution



In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv